In [498]:
# Importing all the dependencies
%load_ext autoreload
%autoreload 2

# Check if running in Colab
in_colab = True

import os
import sys
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from math import sqrt
from sklearn.metrics import mean_absolute_error, r2_score, accuracy_score, classification_report, confusion_matrix
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier, StackingClassifier
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, LogisticRegression
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.svm import SVC, LinearSVC
import xgboost as xgb
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.multioutput import MultiOutputClassifier

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [641]:
df = pd.read_csv('StudentsPerformance.csv')
gender_encoder = LabelEncoder()
test_preparation_encoder = LabelEncoder()
race_ethnicity_encoder = LabelEncoder()
parental_education_encoder = LabelEncoder()
lunch_encoder = LabelEncoder()

df['gender_encoded'] = gender_encoder.fit_transform(df['gender'])
df['test_preparation_encoded'] = test_preparation_encoder.fit_transform(df['test preparation course'])
df['race_ethnicity_encoded'] = race_ethnicity_encoder.fit_transform(df['race/ethnicity'])
df['parental_education_encoded'] = parental_education_encoder.fit_transform(df['parental level of education'])
df['lunch_encoded'] = lunch_encoder.fit_transform(df['lunch'])

df = df.drop(columns=['gender', 'test preparation course', 'race/ethnicity', 'parental level of education', 'lunch'])

In [642]:
x = df.drop(['math score', 'reading score','writing score'], axis=1)
y = df[['math score', 'reading score','writing score']]

In [643]:
x = df.drop(['math score', 'reading score','writing score'], axis=1)
y = df[['math score', 'reading score','writing score']]

In [644]:
df['math pass'] = df['math score'] >= 70
df['reading pass'] = df['reading score'] >= 70
df['writing pass'] = df['writing score'] >= 70

# Convert boolean values to "Pass" and "Fail"
df['math pass'] = df['math pass'].apply(lambda x: 1 if x else 0)
df['reading pass'] = df['reading pass'].apply(lambda x: 'Pass' if x else 'Fail')
df['writing pass'] = df['writing pass'].apply(lambda x: 'Pass' if x else 'Fail')

In [645]:
x = df.drop(['math score', 'reading score','writing score', 'reading pass', 'writing pass'], axis=1)
y = df[['math pass', 'reading pass','writing pass']]

In [646]:
y

,math pass,reading pass,writing pass
0,1,Pass,Pass
1,0,Pass,Pass
2,1,Pass,Pass
3,0,Fail,Fail
4,1,Pass,Pass
...,...,...,...
995,1,Pass,Pass
996,0,Fail,Fail
997,0,Pass,Fail
998,0,Pass,Pass


In [647]:
x = df.drop(['math score', 'reading score','writing score', 'reading pass'], axis=1)
x['writing pass'] = x['writing pass'].apply(lambda x: 1 if x else 0)
y_reading = df[['reading pass']]

In [648]:
y_reading

,reading pass
0,Pass
1,Pass
2,Pass
3,Fail
4,Pass
...,...
995,Pass
996,Fail
997,Pass
998,Pass


In [649]:
x

,gender_encoded,test_preparation_encoded,race_ethnicity_encoded,parental_education_encoded,lunch_encoded,math pass,writing pass
0,0,1,1,1,1,1,1
1,0,0,2,4,1,0,1
2,0,1,1,3,1,1,1
3,1,1,0,0,0,0,1
4,1,1,2,4,1,1,1
...,...,...,...,...,...,...,...
995,0,0,4,3,1,1,1
996,1,1,2,2,0,0,1
997,0,0,2,2,0,0,1
998,0,0,3,4,1,0,1


In [480]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

In [29]:
numerical_columns = ["math score", "reading score", "writing score"]
numerical_data = df[numerical_columns]

scaler = StandardScaler()
scaled_data = scaler.fit_transform(numerical_data)

In [30]:
df_scaled = pd.DataFrame(data=scaled_data, columns=numerical_columns)
df_scaled

,math score,reading score,writing score
0,0.390024,0.193999,0.391492
1,0.192076,1.427476,1.313269
2,1.577711,1.770109,1.642475
3,-1.259543,-0.833899,-1.583744
4,0.653954,0.605158,0.457333
...,...,...,...
995,1.445746,2.044215,1.774157
996,-0.269803,-0.970952,-0.859491
997,-0.467751,0.125472,-0.201079
998,0.126093,0.605158,0.589015


In [560]:
y_math = df[['reading pass', 'writing pass']]
x_train_math, x_test_math, y_train_math, y_test_math = train_test_split(x, y_reading, test_size=0.2, random_state=42)

In [640]:
y_train_math

,reading pass
29,Pass
535,Pass
695,Pass
557,Fail
836,Fail
...,...
106,Pass
270,Fail
860,Fail
435,Fail


In [483]:
x_test_math

,gender_encoded,test_preparation_encoded,race_ethnicity_encoded,parental_education_encoded,lunch_encoded,math pass
521,0,1,2,0,1,1
737,0,0,1,4,0,0
740,1,1,3,1,1,1
660,1,1,2,4,0,1
411,1,0,4,4,1,1
...,...,...,...,...,...,...
408,0,0,3,2,0,0
332,1,0,4,0,1,0
208,0,1,1,4,0,1
613,0,1,2,0,1,0


In [388]:
x_train, x_test, y_train, y_test = train_test_split(x, df_scaled, test_size=0.2, random_state=1)

# **Regression**

## **Random forest**

In [73]:
param_grid_rfg = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'max_features': [3, 5, 8],
}

In [81]:
rfg_model = RandomForestRegressor()
grid_search_rfg = GridSearchCV(estimator=rfg_model, param_grid=param_grid_rfg, cv=5, n_jobs=-1, scoring='r2')

In [84]:
grid_search_rfg.fit(x_test_math, y_test_math)

GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'max_depth': [None, 10, 20, 30],
                         'max_features': [3, 5, 8],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='r2')

In [119]:
best_estimator_rfg = grid_search_rfg.best_estimator_
best_estimator_rfg

RandomForestRegressor(max_depth=30, max_features=3, min_samples_leaf=4,
                      min_samples_split=10)

In [46]:
best_estimator_rfg = RandomForestRegressor(max_depth=30, max_features=3, min_samples_leaf=4,
                      min_samples_split=10)


In [50]:
def rfg(best_estimator, X, Y):
  rfg_r2_train = []
  rfg_r2_test = []
  rfg_mae = []
  folds = 4
  kf = KFold(n_splits=folds)
  for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
    best_estimator.fit(X_train, Y_train)
    Y_train_pred = best_estimator.predict(X_train)
    Y_test_pred = best_estimator.predict(X_test)

    train_r2 = r2_score(Y_train, Y_train_pred)
    test_r2 = r2_score(Y_test, Y_test_pred)

    rfg_r2_train.append(train_r2)
    rfg_r2_test.append(test_r2)
    rfg_mae.append(mean_absolute_error(Y_test, Y_test_pred))

  r2_train_avg = (sum(rfg_r2_train)/len(rfg_r2_train))
  r2_test_avg = (sum(rfg_r2_test)/len(rfg_r2_test))
  mae_avg = (sum(rfg_mae)/len(rfg_mae))

  print(f'R2 Train = {rfg_r2_train}')
  print(f'R2 Test = {rfg_r2_test}')
  print(f'MAE = {rfg_mae}')
  print(f'R2 Train Promedio = {r2_train_avg}')
  print(f'R2 Test Promedio = {r2_test_avg}')
  print(f'MAE Promedio = {mae_avg}')

In [52]:
resultados_rfg = rfg(best_estimator_rfg, x, y_math)
resultados_rfg

R2 Train = [0.3324045026623216, 0.33322732403601085, 0.34858040138105373, 0.33974876482606886]
R2 Test = [0.1710278955091311, 0.18223939753552598, 0.14104885038907367, 0.18576405393142126]
MAE = [11.467500647844231, 10.329971410509064, 11.014512788078399, 11.372102570139763]
R2 Train Promedio = 0.33849024822636375
R2 Test Promedio = 0.170020049341288
MAE Promedio = 11.046021854142865


In [ ]:
num_trees = resultados_rf['número de árboles']
num_variables = resultados_rf['numero de variables']
train_rmse = resultados_rf['train R2']

# Create a 2D plot for train RMSE
plt.figure(figsize=(10, 6))
plt.scatter(num_trees, train_rmse, c=num_variables, cmap='viridis')
plt.colorbar(label='Número de Variables')
plt.xlabel('Número de Árboles')
plt.ylabel('Train R2')
plt.title('Train R2 vs. # of Trees and # of Features')
plt.show()

## **Gradient Boosting**

In [132]:
param_grid_gbt = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'max_features': [1,3,5]
}

In [133]:
gbt_model = GradientBoostingRegressor()
grid_search_gbt = GridSearchCV(estimator=gbt_model, param_grid=param_grid_gbt, cv=5, scoring='r2')

In [139]:
grid_search_gbt.fit(x_train_math, y_train_math)
best_estimator_gbt = grid_search_gbt.best_estimator_
best_estimator_gbt

GradientBoostingRegressor(max_features=1)

In [45]:
best_estimator_gbt = grid_search_gbt.best_estimator_
best_estimator_gbt

NameError: name 'grid_search_gbt' is not defined

In [44]:
y_pred_math = best_estimator_gbt.predict(x_test_math)
test_r2 = r2_score(y_test_math, y_pred_math)
print(test_r2)

NameError: name 'best_estimator_gbt' is not defined

In [ ]:
def gbt(best_estimator, X, Y):
  gbt_r2_train = []
  gbt_r2_test = []
  gbt_mae = []
  folds = 4
  kf = KFold(n_splits=folds)
  for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
    Y_train_pred = best_estimator.predict(X_train)
    Y_test_pred = best_estimator.predict(X_test)

    train_r2 = r2_score(Y_train, Y_train_pred)
    test_r2 = r2_score(Y_test, Y_test_pred)

    gbt_r2_train.append(train_r2)
    gbt_r2_test.append(test_r2)
    gbt_mae.append(mean_absolute_error(Y_test, Y_test_pred))

  r2_train_avg = (sum(gbt_r2_train)/len(gbt_r2_train))
  r2_test_avg = (sum(gbt_r2_test)/len(gbt_r2_test))
  mae_avg = (sum(gbt_mae)/len(gbt_mae))

  print(f'R2 Train = {gbt_r2_train}')
  print(f'R2 Test = {gbt_r2_test}')
  print(f'MAE = {gbt_mae}')
  print(f'R2 Train Promedio = {r2_train_avg}')
  print(f'R2 Test Promedio = {r2_test_avg}')
  print(f'MAE Promedio = {mae_avg}')

In [ ]:
resultados_gbt = gbt(best_estimator_rfg, x, y)
resultados_gbt

## **Redes neuronales artificiales**

In [ ]:
param_grid_mlp = {
    'hidden_layer_sizes': [(50, 50), (100, 100, 100), (50, 100, 50)],
    'activation': ['identity', 'logistic', 'tanh', 'relu'],
    'solver': ['lbfgs', 'adam'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'learning_rate_init': [0.001, 0.01, 0.1],
    'max_iter': [100, 200, 300],
    'early_stopping': [True, False],
}

In [ ]:
mlp_model = MLPRegressor()

grid_search_ann = GridSearchCV(estimator=mlp_model, param_grid=param_grid_mlp, scoring='r2', cv=5, n_jobs=-1)

In [42]:
best_mlp = MLPRegressor(activation='identity', hidden_layer_sizes=(50, 50),learning_rate_init=0.1, max_iter=100, early_stopping=True, alpha=0.001)

In [43]:
# Train the MLP model on the training data
best_mlp.fit(x_train_math, y_train_math)

# Make predictions on the testing data
y_pred = best_mlp.predict(x_test_math)

# Calculate R-squared (R2) score
r2 = r2_score(y_test_math, y_pred)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test_math, y_pred)

# Print the results
print("MLP R2 Score:", r2)
print("MLP MAE:", mae)
print('///////////////////////////////')

MLP R2 Score: 0.17315053407495518
MLP MAE: 11.09624051938716
///////////////////////////////


In [ ]:
grid_search_ann.fit(x,y)
best_estimator_ann = grid_search_ann.best_estimator_
best_estimator_ann

## **Linear reggression**

In [ ]:
x_math = df.drop(['math score', 'reading score','writing score'], axis=1)
y_math = df[['math score']]

x_reading = df.drop(['math score', 'reading score','writing score'], axis=1)
y_reading = df[['reading score']]

x_writing = df.drop(['math score', 'reading score','writing score'], axis=1)
y_writing = df[['writing score']]

In [ ]:
rfg_model_math = RandomForestRegressor()
grid_search_rfg_math = GridSearchCV(estimator=rfg_model_math, param_grid=param_grid_rfg, cv=5, n_jobs=-1, scoring='r2')

In [ ]:
grid_search_rfg_math.fit(x_math, y_math)

In [ ]:
best_estimator_rfg_math = grid_search_rfg_math.best_estimator_
best_estimator_rfg_math

In [ ]:
resultados_rfg_math = rfg(best_estimator_rfg_math, x_math, y_math)
resultados_rfg_math

In [41]:
# Create and train separate linear regression models for each score
lr_math = LinearRegression()
lr_math.fit(x_train_math, y_train_math)


# Make predictions for each score
y_pred_math = lr_math.predict(x_test_math)

# Evaluate the models for each score
mae_math = mean_absolute_error(y_test_math, y_pred_math)
r2_math = r2_score(y_test_math, y_pred_math)


# Print the evaluation results
print("Math Score - Mean Squared Error:", mae_math)
print("Math Score - R-squared (R2) Score:", r2_math)

Math Score - Mean Squared Error: 11.224583244917547
Math Score - R-squared (R2) Score: 0.16629038248152772


In [ ]:
results = []

# Iterate over the scores (math, reading, writing)
for x, y in [(x_math, y_math), (x_reading, y_reading), (x_writing, y_writing)]:
    # Split the data into training and testing sets for the current score
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    
    # Create and train a KNN regression model for the current score
    knn_model = KNeighborsRegressor(n_neighbors=5)  # You can adjust the number of neighbors (n_neighbors) as needed
    knn_model.fit(x_train, y_train)
    
    # Make predictions for the current score
    y_pred = knn_model.predict(x_test)
    
    # Evaluate the model for the current score using MAE
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Append the results to the list
    results.append({
        'MAE': mae,
        'R2': r2
    })

# Print the results for each score
for i, score_name in enumerate(['Math', 'Reading', 'Writing']):
    print(f"{score_name} Score - Mean Absolute Error (MAE):", results[i]['MAE'])
    print(f"{score_name} Score - R-squared (R2) Score:", results[i]['R2'])

## **KNN**

In [ ]:
param_grid_knn = {
    'n_neighbors': [5, 10, 15, 20, 25],
    'p': [1, 2],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [10, 30, 60]
}

In [ ]:
knn_math = KNeighborsRegressor()
grid_search_knn_math = GridSearchCV(estimator=knn_math, param_grid=param_grid_knn, cv=5, n_jobs=-1, scoring='r2')

In [ ]:
grid_search_knn_math.fit(x_math, y_math)

In [ ]:
best_estimator_knn_math = grid_search_knn_math.best_estimator_
best_estimator_knn_math

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_math, y_math, test_size=0.2, random_state=42)

# Make predictions for the training and testing data
y_train_pred = best_estimator_knn_math.predict(X_train)
y_test_pred = best_estimator_knn_math.predict(X_test)

# Calculate R-squared (R2) score and Mean Absolute Error (MAE) for training data
r2_train = r2_score(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)

# Calculate R-squared (R2) score and Mean Absolute Error (MAE) for testing data
r2_test = r2_score(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

# Print the results
print("Training Data - R2 Score:", r2_train)
print("Training Data - MAE:", mae_train)

print("Testing Data - R2 Score:", r2_test)
print("Testing Data - MAE:", mae_test)

In [ ]:
kf = KFold(n_splits=4)

# Perform cross-validation and get R2 scores
r2_scores = cross_val_score(best_estimator_knn_math, x_math, y_math, cv=kf, scoring='r2')

# Perform cross-validation and get MAE scores
mae_scores = -cross_val_score(best_estimator_knn_math, x_math, y_math, cv=kf, scoring='neg_mean_absolute_error')

# Print the cross-validated R2 and MAE scores
print("Cross-Validated R2 Scores:", r2_scores)
print("Mean R2 Score:", r2_scores.mean())

print("Cross-Validated MAE Scores:", mae_scores)
print("Mean MAE:", mae_scores.mean())

## **SVM**

In [ ]:
# Define the SVM parameter grid for tuning
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'epsilon': [0.01, 0.1, 0.2, 0.5, 1],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': [2, 3, 4],
    'gamma': ['scale', 'auto'] + [0.01, 0.1, 1, 10],
    'coef0': [0.0, 1.0, 2.0],
    'shrinking': [True, False],
    'tol': [1e-4, 1e-3, 1e-2],
    'max_iter': [100, 500, 1000]
}


# Create an SVR model
svm_model = SVR()

# Create the GridSearchCV object
svm_math_model = SVR()

# Create GridSearchCV objects for each score
grid_search_math = GridSearchCV(svm_math_model, param_grid, scoring='r2', cv=5, n_jobs=-1)

# Fit the GridSearch for each score
grid_search_math.fit(x_math, y_math)

# Get the best parameters and R2 scores for each score
print("Math Score - Best parameters:", grid_search_math.best_params_)
print("Math Score - Best R2 Score:", grid_search_math.best_score_)
grid_search_math.best_estimator_

# **Classification**

## **Random forest classifier**

In [599]:
param_grid_rfc = {
    'n_estimators': [100, 200, 300],      
    'criterion': ['gini', 'entropy'],    
    'max_depth': [None, 10, 20, 30],     
    'min_samples_split': [2, 5, 10],  
    'max_features': [None, 1, 3, 5],
    'class_weight': [None, 'balanced'] 
}

In [600]:
rfc_model = RandomForestClassifier()
grid_search_rfc = GridSearchCV(estimator=rfc_model, param_grid=param_grid_rfc, cv=5, n_jobs=-1, scoring='accuracy')

In [601]:
grid_search_rfc.fit(x_train_math, y_train_math)

c:\Users\Wilmer Soto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'class_weight': [None, 'balanced'],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 10, 20, 30],
                         'max_features': [None, 1, 3, 5],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='accuracy')

In [602]:
best_estimator_rfc = grid_search_rfc.best_estimator_
best_estimator_rfc

RandomForestClassifier(criterion='entropy', max_depth=30, max_features=1,
                       min_samples_split=10, n_estimators=200)

In [429]:
test_rfc = RandomForestClassifier(criterion='entropy', max_depth=10, max_features=1,min_samples_split=10, n_estimators=300)

test_rfc.fit(x_train_math, y_train_math)

RandomForestClassifier(criterion='entropy', max_depth=10, max_features=1,
                       min_samples_split=10, n_estimators=300)

In [604]:
best_estimator_rfc.fit(x_train_math, y_train_math)

c:\Users\Wilmer Soto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(criterion='entropy', max_depth=30, max_features=1,
                       min_samples_split=10, n_estimators=200)

In [605]:
y_pred = best_estimator_rfc.predict(x_test_math)
math_accuracy = accuracy_score(y_test_math, y_pred)
print(math_accuracy)
print("Math Classification Report:\n", classification_report(y_test_math, y_pred))

0.78
Math Classification Report:
               precision    recall  f1-score   support

        Fail       0.76      0.78      0.77        95
        Pass       0.80      0.78      0.79       105

    accuracy                           0.78       200
   macro avg       0.78      0.78      0.78       200
weighted avg       0.78      0.78      0.78       200



## **KNN**

In [441]:
param_grid_knnc = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'p': [1, 2],
    'leaf_size': [20, 30, 40],
}

In [454]:
knnclass_model = KNeighborsClassifier()
gs_knnclass = GridSearchCV(estimator=knnclass_model, param_grid=param_grid_knnc, cv=5, n_jobs=-1, scoring='accuracy')

In [455]:
gs_knnclass.fit(x_train_math, y_train_math)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'leaf_size': [20, 30, 40],
                         'n_neighbors': [3, 5, 7, 9, 11], 'p': [1, 2],
                         'weights': ['uniform', 'distance']},
             scoring='accuracy')

In [456]:
best_knnc = gs_knnclass.best_estimator_
best_knnc

KNeighborsClassifier(algorithm='brute', leaf_size=20, n_neighbors=11)

In [458]:
best_knnc.fit(x_train_math, y_train_math)

KNeighborsClassifier(algorithm='brute', leaf_size=20, n_neighbors=11)

In [459]:
y_pred = best_knnc.predict(x_test_math)
math_accuracy = accuracy_score(y_test_math, y_pred)
print(math_accuracy)
print("Math Classification Report:\n", classification_report(y_test_math, y_pred))

0.8
Math Classification Report:
               precision    recall  f1-score   support

        Fail       0.85      0.82      0.83       122
        Pass       0.73      0.77      0.75        78

    accuracy                           0.80       200
   macro avg       0.79      0.79      0.79       200
weighted avg       0.80      0.80      0.80       200



## **Naive bayes**

In [113]:
param_grid_nb = {
    'alpha': [0.1, 0.5, 1.0, 1.5],      # Smoothing parameter
    'fit_prior': [True, False],          # Whether to learn class prior probabilities
    'class_prior': [None, [0.3, 0.7], [0.4, 0.6]],  # Specify your own class prior probabilities
}

grid_search_nb = GridSearchCV(MultinomialNB(), param_grid_nb, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_nb.fit(x_train_math, y_train_math)

GridSearchCV(cv=5, estimator=MultinomialNB(), n_jobs=-1,
             param_grid={'alpha': [0.1, 0.5, 1.0, 1.5],
                         'class_prior': [None, [0.3, 0.7], [0.4, 0.6]],
                         'fit_prior': [True, False]},
             scoring='accuracy')

In [118]:
best_nb = grid_search_nb.best_estimator_
best_nb

MultinomialNB(alpha=0.1)

In [123]:
best_nb.fit(x_train_math, y_train_math)
y_math_pred = best_nb.predict(x_test_math)

# Evaluate the model using accuracy or other classification metrics
math_accuracy = accuracy_score(y_test_math, y_math_pred)

# You can also generate a classification report for more details
print(math_accuracy)
print("Math Classification Report:\n", classification_report(y_test_math, y_math_pred))

0.615
Math Classification Report:
               precision    recall  f1-score   support

        Fail       0.63      0.88      0.74       122
        Pass       0.52      0.21      0.29        78

    accuracy                           0.61       200
   macro avg       0.57      0.54      0.51       200
weighted avg       0.59      0.61      0.56       200



## **SVC**

In [503]:
param_grid_svm = {'estimator__C': [1, 10, 100], 
              'estimator__gamma': [1,0.1,0.01],
              'estimator__kernel': ['rbf', 'poly', 'sigmoid'],
              'estimator__probability' : [True]}

svm = SVC()
grid_search_svc = GridSearchCV(svm, param_grid_svm, cv=5, scoring='accuracy', n_jobs=-1)

In [ ]:
grid_search_svc.fit(x_train_math, y_train_math)

In [228]:
best_svm = grid_search_svc.best_estimator_
best_svm

SVC(C=100, gamma=0.001, kernel='sigmoid')

In [212]:
best_svm.fit(x_train_math, y_train_math)

SVC(C=100, gamma=0.001, kernel='sigmoid')

In [596]:
test_svm = SVC(C=100, gamma=0.001, kernel='sigmoid', probability=True)
test_svm.fit(x_train_math, y_train_math)

c:\Users\Wilmer Soto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=100, gamma=0.001, kernel='sigmoid', probability=True)

In [597]:
y_pred = test_svm.predict(x_test_math)
accuracy = accuracy_score(y_test_math, y_pred)
classification = classification_report(y_test_math, y_pred)
print(accuracy)
print("Classification Report:\n")
print(classification)

0.775
Classification Report:

              precision    recall  f1-score   support

        Fail       0.70      0.91      0.79        95
        Pass       0.88      0.66      0.75       105

    accuracy                           0.78       200
   macro avg       0.79      0.78      0.77       200
weighted avg       0.80      0.78      0.77       200



In [230]:
y_pred = best_svm.predict(x_test_math)
accuracy = accuracy_score(y_test_math, y_pred)
classification = classification_report(y_test_math, y_pred)
print(accuracy)
print("Classification Report:\n")
print(classification)

0.695
Classification Report:

              precision    recall  f1-score   support

        Fail       0.75      0.75      0.75       122
        Pass       0.61      0.62      0.61        78

    accuracy                           0.69       200
   macro avg       0.68      0.68      0.68       200
weighted avg       0.70      0.69      0.70       200



## **LinearSVC**

In [528]:
param_grid_svc = {
    'C': [0.01, 0.1, 1.0, 10.0],  # Regularization parameter
    'loss': ['hinge', 'squared_hinge'],  # Loss function
    'tol': [1e-4, 1e-3, 1e-2],# Tolerance for stopping criteria
}

In [529]:
svm_math = LinearSVC()
grid_search_lsvc = GridSearchCV(estimator=svm_math, param_grid=param_grid_svc, cv=5, scoring='accuracy', n_jobs=-1)

In [530]:
grid_search_lsvc.fit(x_train_math, y_train_math)

c:\Users\Wilmer Soto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Wilmer Soto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\Wilmer Soto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


GridSearchCV(cv=5, estimator=LinearSVC(), n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1.0, 10.0],
                         'loss': ['hinge', 'squared_hinge'],
                         'tol': [0.0001, 0.001, 0.01]},
             scoring='accuracy')

In [531]:
best_lsvc = grid_search_lsvc.best_estimator_
best_lsvc

LinearSVC(C=10.0)

In [532]:
best_lsvc.fit(x_train_math, y_train_math)

c:\Users\Wilmer Soto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Wilmer Soto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\Wilmer Soto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(C=10.0)

In [533]:
y_pred = best_lsvc.predict(x_test_math)
math_accuracy = accuracy_score(y_test_math, y_pred)
print(math_accuracy)
print("Math Classification Report:\n", classification_report(y_test_math, y_pred))

0.805
Math Classification Report:
               precision    recall  f1-score   support

        Fail       0.79      0.81      0.80        95
        Pass       0.82      0.80      0.81       105

    accuracy                           0.81       200
   macro avg       0.80      0.81      0.80       200
weighted avg       0.81      0.81      0.81       200



## **MLP Classifier**

In [534]:
param_grid_mlpc = {
    'hidden_layer_sizes': [(50, 50), (100, 100)],
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'early_stopping': [True, False],
}

In [535]:
mlp_classmodel = MLPClassifier()
gs_mlpclass = GridSearchCV(estimator=mlp_classmodel, param_grid=param_grid_mlpc, cv=5, n_jobs=-1, scoring='accuracy')

In [536]:
gs_mlpclass.fit(x_train_math, y_train_math)

c:\Users\Wilmer Soto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1102: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=5, estimator=MLPClassifier(), n_jobs=-1,
             param_grid={'activation': ['logistic', 'tanh', 'relu'],
                         'early_stopping': [True, False],
                         'hidden_layer_sizes': [(50, 50), (100, 100)],
                         'learning_rate': ['constant', 'invscaling',
                                           'adaptive'],
                         'solver': ['lbfgs', 'sgd', 'adam']},
             scoring='accuracy')

In [537]:
gs_mlpclass.best_estimator_

MLPClassifier(activation='tanh', hidden_layer_sizes=(50, 50),
              learning_rate='adaptive')

In [538]:
best_mlpc = gs_mlpclass.best_estimator_.fit(x_train_math, y_train_math)

c:\Users\Wilmer Soto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1102: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Wilmer Soto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [539]:
y_math_pred = best_mlpc.predict(x_test_math)
math_accuracy = accuracy_score(y_test_math, y_math_pred)
print(math_accuracy)
print("Math Classification Report:\n", classification_report(y_test_math, y_math_pred))

0.8
Math Classification Report:
               precision    recall  f1-score   support

        Fail       0.78      0.81      0.79        95
        Pass       0.82      0.79      0.81       105

    accuracy                           0.80       200
   macro avg       0.80      0.80      0.80       200
weighted avg       0.80      0.80      0.80       200



## **Gaussian NB**

In [591]:
param_grid_gnb = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3],
}

In [592]:
gnb_classmodel = GaussianNB()
gs_gnbclass = GridSearchCV(estimator=gnb_classmodel, param_grid=param_grid_gnb, cv=5, n_jobs=-1, scoring='accuracy')

In [593]:
gs_gnbclass.fit(x_train_math, y_train_math)

c:\Users\Wilmer Soto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=5, estimator=GaussianNB(), n_jobs=-1,
             param_grid={'var_smoothing': [1e-09, 1e-08, 1e-07, 1e-06, 1e-05,
                                           0.0001, 0.001]},
             scoring='accuracy')

In [594]:
gs_gnbclass.best_score_
best_gnb = gs_gnbclass.best_estimator_
best_gnb

GaussianNB()

In [595]:
best_gnb.fit(x_train_math, y_train_math)
y_math_pred = best_gnb.predict(x_test_math)
math_accuracy = accuracy_score(y_test_math, y_math_pred)
print(math_accuracy)
print("Math Classification Report:\n", classification_report(y_test_math, y_math_pred))

0.795
Math Classification Report:
               precision    recall  f1-score   support

        Fail       0.75      0.85      0.80        95
        Pass       0.85      0.74      0.79       105

    accuracy                           0.80       200
   macro avg       0.80      0.80      0.79       200
weighted avg       0.80      0.80      0.79       200



c:\Users\Wilmer Soto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [80]:
test_rfc =RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features=5, max_leaf_nodes=None,
            min_impurity_decrease=0.0,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

test_rfc.fit(x_train_math, y_train_math)
y_pred = test_rfc.predict(x_test_math)
print('accuracy score:',accuracy_score(y_test_math, y_pred)) 

accuracy score: 0.635


## **Ensemble**

### **Gradient Boosting**

In [615]:
param_grid_gbc = {
    'n_estimators': [100, 200, 300],        # Number of boosting stages to be used
    'learning_rate': [0.01, 0.1, 0.2],    # Step size shrinkage used in update to prevent overfitting
    'max_depth': [3, 4, 5],              # Maximum depth of the individual trees
    'min_samples_split': [2, 5, 10],     # Minimum samples required to split an internal node
    'subsample': [0.8, 0.9, 1.0],        # Fraction of samples used for fitting the trees
    'max_features': [None, 1, 3, 5],  # Number of features to consider for the best split
}

In [616]:
gb_classifier = GradientBoostingClassifier()
gs_gbc = GridSearchCV(estimator=gb_classifier, param_grid=param_grid_gbc, cv=5, n_jobs=-1, scoring='accuracy')

In [617]:
gs_gbc.fit(x_train_math, y_train_math)

c:\Users\Wilmer Soto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 4, 5],
                         'max_features': [None, 1, 3, 5],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300],
                         'subsample': [0.8, 0.9, 1.0]},
             scoring='accuracy')

In [618]:
best_gbc = gs_gbc.best_estimator_
best_gbc

GradientBoostingClassifier(learning_rate=0.01, max_depth=4, max_features=3,
                           subsample=0.8)

In [620]:
best_gbc.fit(x_train_math, y_train_math)

c:\Users\Wilmer Soto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(learning_rate=0.01, max_depth=4, max_features=3,
                           subsample=0.8)

In [621]:
y_pred = best_gbc.predict(x_test_math)
math_accuracy = accuracy_score(y_test_math, y_pred)
print(math_accuracy)
print("Math Classification Report:\n", classification_report(y_test_math, y_pred))

0.795
Math Classification Report:
               precision    recall  f1-score   support

        Fail       0.78      0.79      0.79        95
        Pass       0.81      0.80      0.80       105

    accuracy                           0.80       200
   macro avg       0.79      0.79      0.79       200
weighted avg       0.80      0.80      0.80       200



### **Adaboost**

In [608]:
param_grid_ada = {
    'base_estimator': [None],  # You can specify the base estimator (e.g., DecisionTreeClassifier) here
    'n_estimators': [50, 100, 200],  # Number of boosting stages
    'learning_rate': [0.01, 0.1, 1.0],  # Step size shrinkage
    'algorithm': ['SAMME', 'SAMME.R'],  # Algorithm used for boosting
}

In [609]:
ada_model = AdaBoostClassifier()
grid_search_ada = GridSearchCV(estimator=ada_model, param_grid=param_grid_ada, cv=5, scoring='accuracy', n_jobs=-1)

In [610]:
grid_search_ada.fit(x_train_math, y_train_math)

c:\Users\Wilmer Soto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Wilmer Soto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


GridSearchCV(cv=5, estimator=AdaBoostClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['SAMME', 'SAMME.R'],
                         'base_estimator': [None],
                         'learning_rate': [0.01, 0.1, 1.0],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy')

In [611]:
best_ada = grid_search_ada.best_estimator_
best_ada

AdaBoostClassifier(algorithm='SAMME', base_estimator=None, learning_rate=0.1,
                   n_estimators=200)

In [613]:
best_ada.fit(x_train_math, y_train_math)

c:\Users\Wilmer Soto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Wilmer Soto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


AdaBoostClassifier(algorithm='SAMME', base_estimator=None, learning_rate=0.1,
                   n_estimators=200)

In [614]:
y_pred = best_ada.predict(x_test_math)
math_accuracy = accuracy_score(y_test_math, y_pred)
print(math_accuracy)
print("Math Classification Report:\n", classification_report(y_test_math, y_pred))

0.8
Math Classification Report:
               precision    recall  f1-score   support

        Fail       0.78      0.80      0.79        95
        Pass       0.82      0.80      0.81       105

    accuracy                           0.80       200
   macro avg       0.80      0.80      0.80       200
weighted avg       0.80      0.80      0.80       200



### **XGBoost**

In [555]:
param_grid_xgb = {
    'n_estimators': [100, 200, 300],  # Number of boosting stages (trees)
    'learning_rate': [0.01, 0.1, 0.2],  # Step size shrinkage
    'max_depth': [3, 4, 5],  # Maximum depth of individual trees
    'min_child_weight': [1, 2, 3],  # Minimum sum of instance weight (hessian) needed in a child
    'subsample': [0.8, 0.9, 1.0],  # Fraction of samples used for fitting trees
}

In [556]:
xgb_classifier = xgb.XGBClassifier()
grid_search_xgb = GridSearchCV(estimator=xgb_classifier, param_grid=param_grid_xgb, cv=5, scoring='accuracy', n_jobs=-1)

In [579]:
y_train_math_xgb = y_train_math.replace({'Pass': 1, 'Fail': 0})
y_test_math_xgb = y_test_math.replace({'Pass': 1, 'Fail': 0})

In [580]:
y_train_math_xgb

,reading pass
29,1
535,1
695,1
557,0
836,0
...,...
106,1
270,0
860,0
435,0


In [584]:
grid_search_xgb.fit(x_train_math, y_train_math_xgb)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 4, 5], 'min_child_weight': [1, 2, 3],
                         'n_estimators': [100, 200, 300],
                         'subsample': [0.8, 0.9, 1.0]},
             scoring='accuracy')

In [586]:
gs_xgb = grid_search_xgb.best_estimator_.fit(x_train_math, y_train_math_xgb)

In [587]:
gs_xgb.fit(x_train_math, y_train_math_xgb)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [590]:
y_pred = gs_xgb.predict(x_test_math)
math_accuracy = accuracy_score(y_test_math_xgb, y_pred)
print(math_accuracy)
print("Math Classification Report:\n", classification_report(y_test_math_xgb, y_pred))

0.79
Math Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.79      0.78        95
           1       0.81      0.79      0.80       105

    accuracy                           0.79       200
   macro avg       0.79      0.79      0.79       200
weighted avg       0.79      0.79      0.79       200



### **Voting**

In [606]:
c1 = test_svm
c2 = best_estimator_rfc
c3 = best_mlpc
c4 = best_gnb

for c in [c1, c2, c3, c4]:
    y_pred = c.predict(x_test_math)
    math_accuracy = accuracy_score(y_test_math, y_pred)
    print(f'accuracy = {math_accuracy}')

voting_classifier = VotingClassifier(
    estimators=[('svm', c1), ('rfc', c2), ('mlpc', c3), ('gnb', c4)],
    voting='hard'
)

voting_classifier.fit(x_train_math, y_train_math)

y_pred = voting_classifier.predict(x_test_math)

# Evaluate the performance
accuracy = accuracy_score(y_test_math, y_pred)
print(f"Accuracy of Voting Classifier: {accuracy:.2f}")

accuracy = 0.775
accuracy = 0.78
accuracy = 0.8
accuracy = 0.795


c:\Users\Wilmer Soto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Wilmer Soto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Accuracy of Voting Classifier: 0.81


### **Stacking**

In [607]:
c1 = test_svm
c2 = best_estimator_rfc
c3 = best_mlpc
c4 = best_gnb

base_classifiers = [
    ('svm', c1),
    ('rfc', c2),
    ('mlpc', c3),
    ('gnb', c4)
]

stacking = StackingClassifier(estimators=base_classifiers, final_estimator=LogisticRegression())

stacking.fit(x_train_math, y_train_math)

y_pred = voting_classifier.predict(x_test_math)

accuracy = accuracy_score(y_test_math, y_pred)
print(f"Accuracy of Voting Classifier: {accuracy:.2f}")

c:\Users\Wilmer Soto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Wilmer Soto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Accuracy of Voting Classifier: 0.81


## **QDA**

In [622]:
param_grid_qda = {
    'reg_param': [0.01, 0.1,0.5, 1.0]  # You can adjust the values and range
}

In [623]:
qda_classifier = QuadraticDiscriminantAnalysis()
grid_search_qda = GridSearchCV(estimator=qda_classifier, param_grid=param_grid_qda, cv=5, scoring='accuracy', n_jobs=-1)

In [628]:
grid_search_qda.fit(x_train_math, y_train_math)

c:\Users\Wilmer Soto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Wilmer Soto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


GridSearchCV(cv=5, estimator=QuadraticDiscriminantAnalysis(), n_jobs=-1,
             param_grid={'reg_param': [0.01, 0.1, 0.5, 1.0]},
             scoring='accuracy')

In [629]:
best_qda = grid_search_qda.best_estimator_
best_qda

QuadraticDiscriminantAnalysis(reg_param=0.01)

In [631]:
best_qda.fit(x_train_math, y_train_math)

c:\Users\Wilmer Soto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Wilmer Soto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QuadraticDiscriminantAnalysis(reg_param=0.01)

In [632]:
y_pred = best_qda.predict(x_test_math)
math_accuracy = accuracy_score(y_test_math, y_pred)
print(math_accuracy)
print("Math Classification Report:\n", classification_report(y_test_math, y_pred))

0.795
Math Classification Report:
               precision    recall  f1-score   support

        Fail       0.76      0.82      0.79        95
        Pass       0.83      0.77      0.80       105

    accuracy                           0.80       200
   macro avg       0.80      0.80      0.79       200
weighted avg       0.80      0.80      0.80       200

